In [ ]:
54.243455533342086

In [1]:
path = "/Users/rishabjain/Desktop/Research/choformer/benchmarking/CHO High Exp.csv"

import pandas as pd
import numpy as np

df = pd.read_csv(path)
print(df.head())

                                                 dna
0  ATGCGGAGGTCGCGACCGGCGCGGCGGGTCTCCCTGGATTTAAGCC...
1  ATGAGCTTTTTAGGTGGGTGGAGAAAGGAAGGAAGAAAGGAGGAAG...
2  ATGGCCACTCTGTCCCCATTCATGGCTCTCAAGTCCCTCCCCATTT...
3  ATGGGGTGTGAGTTCCCTTCCCAGAGTGCCTGTGGCGGTGACAGGA...
4  ATGTCTGTTGAGCACAAGAAGGAATTAAGTGACACTTATACTCTGA...


In [4]:
from collections import defaultdict

# Genetic code dictionary
codon_table = {
    'F': ['TTT', 'TTC'], 'L': ['TTA', 'TTG', 'CTT', 'CTC', 'CTA', 'CTG'], 'I': ['ATT', 'ATC', 'ATA'],
    'M': ['ATG'], 'V': ['GTT', 'GTC', 'GTA', 'GTG'], 'S': ['TCT', 'TCC', 'TCA', 'TCG', 'AGT', 'AGC'],
    'P': ['CCT', 'CCC', 'CCA', 'CCG'], 'T': ['ACT', 'ACC', 'ACA', 'ACG'], 'A': ['GCT', 'GCC', 'GCA', 'GCG'],
    'Y': ['TAT', 'TAC'], 'H': ['CAT', 'CAC'], 'Q': ['CAA', 'CAG'], 'N': ['AAT', 'AAC'],
    'K': ['AAA', 'AAG'], 'D': ['GAT', 'GAC'], 'E': ['GAA', 'GAG'], 'C': ['TGT', 'TGC'],
    'W': ['TGG'], 'R': ['CGT', 'CGC', 'CGA', 'CGG', 'AGA', 'AGG'], 'G': ['GGT', 'GGC', 'GGA', 'GGG'],
    'STOP': ['TAA', 'TAG', 'TGA']
}

# Function to count codons in a DNA sequence
def count_codons(dna_seq):
    codon_count = defaultdict(int)
    for i in range(0, len(dna_seq) - 2, 3):
        codon = dna_seq[i:i+3].upper()
        if codon in codon_table['STOP']:
            continue
        codon_count[codon] += 1
    return codon_count

# Function to calculate ENC
def calculate_enc(dna_seq):
    codon_count = count_codons(dna_seq)
    amino_acid_count = defaultdict(int)
    codon_usage_per_aa = defaultdict(lambda: defaultdict(int))
    
    # Count codon usage per amino acid
    for aa, codons in codon_table.items():
        for codon in codons:
            if codon in codon_count:
                codon_usage_per_aa[aa][codon] = codon_count[codon]
                amino_acid_count[aa] += codon_count[codon]

    # Calculate the number of codons per amino acid group
    enc = 0.0
    num_amino_acids = 0

    for aa, codons in codon_usage_per_aa.items():
        if amino_acid_count[aa] == 0:
            continue
        num_amino_acids += 1
        # Calculate the number of synonymous codons
        num_codons_for_aa = len(codon_table[aa])
        F = sum((count / amino_acid_count[aa]) ** 2 for count in codons.values())
        if num_codons_for_aa > 1:
            enc += num_codons_for_aa * (1 - F) / (num_codons_for_aa - 1)
        else:
            enc += 1  # For amino acids encoded by one codon, ENC is 1

    # Normalize the ENC value to range from 20 to 61
    if num_amino_acids > 0:
        enc = num_amino_acids + enc
    else:
        enc = 61  # If no amino acids were counted, default to maximum ENC

    return enc

# Test sequence (replace with your own DNA sequence)
dna_sequence = "ATGCGTACTGAACGTAGTGCTAGCTGATCGACTACGTAGCATCGTACTGAGCGTACGTTG"

enc_value = calculate_enc(dna_sequence)
print(f"Effective Number of Codons (ENC): {enc_value}")

Effective Number of Codons (ENC): 11.44
